In [10]:
import starepandas
import geopandas
import sqlalchemy

# Data Prep

In [11]:
cities = ['Buenos Aires', 'Brasilia', 'Santiago', 
          'Bogota', 'Caracas', 'Sao Paulo', 'Bridgetown']

latitudes = [-34.58, -15.78, -33.45, 4.60, 10.48, -23.55, 13.1]
longitudes = [-58.66, -47.91, -70.66, -74.08, -66.86, -46.63, -59.62]
data =  {'City': cities, 
         'Latitude': latitudes, 'Longitude': longitudes}

cities = starepandas.STAREDataFrame(data)
stare = starepandas.stare_from_xy(cities.Longitude, cities.Latitude, level=27)
cities.set_stare(stare, inplace=True)
cities

,City,Latitude,Longitude,stare
0,Buenos Aires,-34.58,-58.66,2663379193440875387
1,Brasilia,-15.78,-47.91,2867415364672350651
2,Santiago,-33.45,-70.66,2723774768829278555
3,Bogota,4.60,-74.08,2667981979956219515
4,Caracas,10.48,-66.86,2494081632617553403
5,Sao Paulo,-23.55,-46.63,2803225788975740475
6,Bridgetown,13.10,-59.62,2518254660685127707


# To SQLite

In [12]:
db_path = '' # Empty string for inmemory
uri = 'sqlite:///{db_path}'.format(db_path=db_path)
engine = sqlalchemy.create_engine(uri)
cities.to_sql(name='cities', con=engine, if_exists='replace',)

# To ScidDB

In [13]:
import scidbpy
db = scidbpy.db.DB(scidb_url='http://schiss.eri.ucsb.edu:8080/')

In [14]:
schema = '<Latitude: float, stare: int64>[idx]'
records = cities[['Latitude', 'stare']].to_records(index=False)
a = db.input(schema, upload_data=records).store('cities')

In [15]:
cities.dtypes

City          object
Latitude     float64
Longitude    float64
stare          int64
dtype: object

In [16]:
query = 'scan({array})'.format(array='test')
db.iquery(query=query, fetch=True)

/home/griessbaum/.virtualenvs/starepandas/lib/python3.8/site-packages/scidbpy/schema.py:747: UserWarning: 2 type(s) promoted for null support. Precision loss may occur
  self._promo_warning()


,idx,Latitude,stare
0,0,-34.580002,2.663379e+18
1,1,-15.780000,2.867415e+18
2,2,-33.450001,2.723775e+18
3,3,4.600000,2.667982e+18
4,4,10.480000,2.494082e+18
5,5,-23.549999,2.803226e+18
6,6,13.100000,2.518255e+18


In [17]:
query = "remove({array})".format(array=name)
db.iquery(query=query, fetch=False)

NameError: name 'name' is not defined